# Accessing PHE Covid Data

[Public Health England](https://www.gov.uk/government/organisations/public-health-england) (PHE) is one of many Institutions worldwide running a Coronavirus [dashboard](https://coronavirus.data.gov.uk/), with current statistics on the pandemic. In this series of notebooks, we will guide you through creating your own simple dashboard based on PHE data and putting it online as a [Binder](https://mybinder.org/).

You want to start by clicking on *Developer's<sup>1</sup> Guide* in the dashboard menu, which will take you straight to the  [API documentation](https://coronavirus.data.gov.uk/developers-guide). This will be your main source of information for this project, as far as the interaction with the PHE servers is concerned.

<sup>1</sup> *Developer*: that's you!

## The web-based API and the SDK

Many websites support access to their underlying data through a web-based [Application Programming Interface](https://en.wikipedia.org/wiki/API) (an API for short). This is often based on the *http* protocol, and may involve the excange of information in [JSON format](https://en.wikipedia.org/wiki/JSON). Specifically, using a web-based API typically involves sending *http* requests with parameters conforming to a given schema to a dedicated URL (the API *endpoint*), to which the server responds with JSON content. All this is specified in the PHE [API documentation](https://coronavirus.data.gov.uk/developers-guide).

As you can see from the documentation, PHE actually also offers a Python Software Devekopment Kit (SDK). This is a Python wrapper that facilitates access to the API by building the requests for you, forwarding them to the API access point and packaging the response into some convenient format. The PHE [Pyhton SDK](https://pypi.org/project/uk-covid19/) is found on the standard Python package repository, PyPi.

The first step, therefore, consists in installing the Python SDK. This will depend on how you are accessing these matierials.

* If you are working in a Binder: the SDK is already installed for you.
* On the EECS JupyterHub: Save your notebooks and close them. Open a terminal and enter the following command (note the double minus ```--```):
```
pip install --user uk-covid19
```
you can then close the terminal (type ```exit``` and close the tab). Finally go to *Control Panel > Stop my Server*, start the server again and reopen your notebooks.
* On your local machine: this depends on your setup. Entering this line in a command shell will hopefully work:
```
pip install uk-covid19
```
if it doesn't, try searching the documentation of your Python distribution or Google for instructions on how to install PiPy packages on your machine. 

If you have successfully  installed the SDK in your environment, the following cell should work:

In [5]:
# note the name of the module has an underscore in place of the -
from uk_covid19 import Cov19API
import json

## Accessing the API through the SDK

##### We are now ready to download data from the server via the SDK. Before continuing, you should have a look at the [SDK documentation](https://publichealthengland.github.io/coronavirus-dashboard-api-python-sdk/index.html), which will refer you to the general [API documentation](https://coronavirus.data.gov.uk/developers-guide) for the meaning of the main parameters. The examples below follow the documentation closely.

According to the documentation, the first step is defining a [*filter*](https://publichealthengland.github.io/coronavirus-dashboard-api-python-sdk/pages/examples/general_use.html#filters) - this is a **list** specifiying an *areaType* parameter and an optional *areaName*, *areaCode* and *date* parameters. In the jargon of the documentation, these parameters are called *metrics* - check out the metrics allowed for filters [here](https://coronavirus.data.gov.uk/developers-guide#params-filters).

In [6]:
filters = [
    'areaType=overview' # note each metric-value pair is inside one string
]

Next, you want do define a [*structure*](https://publichealthengland.github.io/coronavirus-dashboard-api-python-sdk/pages/examples/general_use.html#structure). According to the SDK ocumentation, this is a **dictionary** that specifies which data fields you want to request. In fact, looking at the API documentation for [structures](https://coronavirus.data.gov.uk/developers-guide#params-structure) shows that they do more: they also specify the "format" in which you want to receive the response. Read the documentation carefully, including the list of valid *metrics* for a *structure* (which is in fact the most important part).

The structure below selects the metrics (that is, the PHE database fields) given as values of the dictionary, and at the same time instructs the API to rename the fields to the simpler names given as keys before serving them to us.

In [7]:
# values here are the names of the PHE metrics
structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "cumDeaths28DaysByDeathDateRate"    
}

**NOTE:** As of version 1.2.0 of the SDK, keys in the ```structure``` dictionary are apparently restricted to alphabet letters. Spaces, digits and underscores seem to cause the query to fail and should be avoided.

Accessing the API is easy at this point. You just need to create a ```Cov19API``` object by passing the filters and structure to its constructor, as follows:

In [8]:
api = Cov19API(filters=filters, structure=structure)

Finally, calling the ```get_json()``` method of the ```api``` object actually sends the request to the API and retrieves the response (theoretically) in JSON format.

In [9]:
# NOTE: this call polls the server. It may fail in case of connectivity problems or if the data
# are not available for any reason. It will also fail if the metric in the structure are not compatible
# with the filters (eg they are not defined at the national or local level).
timeseries=api.get_json()

In [10]:
print(timeseries)

{'data': [{'date': '2020-11-24', 'cases': 11299, 'hospital': None, 'deaths': None}, {'date': '2020-11-23', 'cases': 15450, 'hospital': None, 'deaths': 83.6}, {'date': '2020-11-22', 'cases': 18662, 'hospital': None, 'deaths': 83.4}, {'date': '2020-11-21', 'cases': 19875, 'hospital': None, 'deaths': 83}, {'date': '2020-11-20', 'cases': 20252, 'hospital': 1506, 'deaths': 82.7}, {'date': '2020-11-19', 'cases': 22915, 'hospital': 1648, 'deaths': 82.2}, {'date': '2020-11-18', 'cases': 19609, 'hospital': 1754, 'deaths': 81.6}, {'date': '2020-11-17', 'cases': 20051, 'hospital': 1686, 'deaths': 80.9}, {'date': '2020-11-16', 'cases': 21363, 'hospital': 1740, 'deaths': 80.3}, {'date': '2020-11-15', 'cases': 24962, 'hospital': 1649, 'deaths': 79.7}, {'date': '2020-11-14', 'cases': 26860, 'hospital': 1598, 'deaths': 79.1}, {'date': '2020-11-13', 'cases': 27301, 'hospital': 1623, 'deaths': 78.5}, {'date': '2020-11-12', 'cases': 33470, 'hospital': 1892, 'deaths': 77.9}, {'date': '2020-11-11', 'cases'

In [25]:
print(type(timeseries)) # hold on, this is not JSON!

<class 'dict'>


As you can see, ```get_json()``` is a bit of a misnomer - the function actually returns a dictionary containing nested lists and dictionaries. As we will see this is not too different from what a JSON string looks like, but technically, this is something different. All the better for us, we don't need to decode it - but we may actually want to encode it, see below.

Also, some entries contain a value of ```None```, we will asssume that stands for a ```0``` rather than for not available. In any case, by and large we got our data, so this is a success.

### Another example: Cases by gender and age

The example above lends itself to visualisation as a plot of cases, hospital admissions and fatalities vs time. In this example, instead, we investigate the distribution of cases by sex and age bands; eventually, we will plot this as a bar chart.

Again we define our *filters* and *structure*, but with different *metrics*, as follows: 

In [21]:
filters = [
    'areaType=nation',
    'areaName=Wales'
]


# values here are the names of the PHE metrics
structure = {
    "males": "maleCases",
    "females": "femaleCases"
}

The next two steps are standard:

In [22]:
api = Cov19API(filters=filters, structure=structure)

In [23]:
# NOTE: this call polls the server. It may fail in case of connectivity problems or if the data
# are not available for any reason. It will also fail if the metric in the structure are not compatible
# with the filters (eg they are not defined at the national or local level).
agedistribution = api.get_json()

In [24]:
print(agedistribution)

{'data': [{'males': [{'age': '0_to_4', 'value': 578, 'rate': 682.4}, {'age': '5_to_9', 'value': 569, 'rate': 537.5}, {'age': '10_to_14', 'value': 920, 'rate': 976.6}, {'age': '15_to_19', 'value': 2522, 'rate': 2720.2}, {'age': '20_to_24', 'value': 3394, 'rate': 3800.7}, {'age': '25_to_29', 'value': 2756, 'rate': 2591.6}, {'age': '30_to_34', 'value': 2531, 'rate': 2375.6}, {'age': '35_to_39', 'value': 2266, 'rate': 2353.9}, {'age': '40_to_44', 'value': 2157, 'rate': 2359.6}, {'age': '45_to_49', 'value': 2438, 'rate': 2914.1}, {'age': '50_to_54', 'value': 2514, 'rate': 2592.3}, {'age': '55_to_59', 'value': 2556, 'rate': 2382.3}, {'age': '60_to_64', 'value': 1741, 'rate': 1854.1}, {'age': '65_to_69', 'value': 1167, 'rate': 1317.7}, {'age': '70_to_74', 'value': 1122, 'rate': 1295.1}, {'age': '75_to_79', 'value': 1129, 'rate': 1897.4}, {'age': '80_to_84', 'value': 1023, 'rate': 2576.8}, {'age': '85_to_89', 'value': 786, 'rate': 3673.6}, {'age': '90+', 'value': 572, 'rate': 5902.4}, {'age': 

Again this is a dictionary and it does contain the data we expected. Note however how the formatting and the type of data depend, maybe unsurprisingly, on the specific *metric* we queried.

## Saving the data in JSON format

At this point, we want to save the result of our API queries in order to 
* have something definite to work on in the other notebooks
* eventually, give our dashboard some starting data.

The problem arises of how to save these dictionaries to the disk. Luckily we do not have to save them in a bespoke way at this stage - we can use the [json module](https://docs.python.org/3/library/json.html) in the stardard library to dump them sa they are in [JSON format](https://en.wikipedia.org/wiki/JSON). This is straightforward:

In [30]:
import json

In [31]:
with open("timeseries.json", "wt") as OUTF:
    json.dump(timeseries, OUTF)

In [16]:
with open("agedistribution.json", "wt") as OUTF:
    json.dump(agedistribution, OUTF)

If you now use a text editor (or the Jupyter Notebook interface), you will see that the content of the files closely resembles the tangle of dictionaries and lists we have seen above. However, technically, these are no longer Python dictionary and files, rather the JSON representation of them, and could be opened by another program written in another language, that will map them to an equivalent data structure (whichever is provided by that language).

## Your turn

Explore the various *metrics* available for [*filters*](https://coronavirus.data.gov.uk/developers-guide#params-filters) and [*structures*](https://coronavirus.data.gov.uk/developers-guide#params-structure) and think of a query that may be of interest to you, and how you might then want to visualise the data. You can modify either the *structure*, in order to select different types of data, or the *filters*, to specify a different granularity (national or local level, specific dates, etc). Possible graphs of interest might be:
* a comparison of the number of test carried out with planned testing capacity;
* a comparison of the number of new cases with the number of tests;
* the above, broken down by region;
* a comparison of the number of cases with hospital admissions;
* a comparison of hospital admissions with ventilator bed occupancy;
* new cases in major cities as a fraction of the population of those cities;
* a comparison of the age distribution of new cases at different times (this will require more than one API access)

Please keep in mind the following points:
* Not all metrics are available for all dates, or at all levels of granularity; querying for data that's unavailable will result in an error.
* Documentation is poor - welcome to the real world. A BSc in Reverse Engineering would come in handy.
* Experimenting is fine. However, avoid flooding the server with multiple queries at machine speed - if you use a ```for``` loop to generate API accesses, use the ```sleep()``` function from the ```time``` module to introduce a 1 second delay between one query and the next (see [here](https://docs.python.org/3/library/time.html#time.sleep) for details). The last thing you want is for PHE to ban you.

Once you succeed in retrieving the data you want, save them in JSON format and move on to the next stage - visualisation.


**NOTE:** In the visualisation stage, we are going to load the data into a ```pandas.DataFrame``` structure. As you will see from the documentation, the SDK contains a method ```get_dataframe()``` that allows you to retrieve the data in the form of a ```DataFrame``` directly. From my tests, however, this method only works with certain *metrics*, and returns gibberish in other cases. I would therefore advise you to stick with JSON. In any case, JSON is a de-facto standard for APIs, and some familiarity with this format is a valuable skill in itself.

In [3]:
from uk_covid19 import Cov19API
import json

filters = [
    'areaType=overview'
    'areaName=England'
]

structure = {
   
}

api = Cov19API(filters=filters, structure=structure)

agedistribution=api.get_json()

print(agedistribution)

FailedRequestError: 
Request failed .... 500 - Internal Server Error
Response .......... {"response":"An internal error occurred whilst processing your request, please try again. If the problem persists, please report as an issue and include your request.","status_code":500,"status":"Internal Server Error"}
URL ............... https://api.coronavirus.data.gov.uk/v1/data?filters=areaType%3DoverviewareaName%3DEngland&structure=%7B%22males%22%3A%22maleCases%22%2C%22females%22%3A%22femaleCases%22%7D&format=json&page=1
Decoded URL ....... https://api.coronavirus.data.gov.uk/v1/data?filters=areaType=overviewareaName=England&structure={"males":"maleCases","females":"femaleCases"}&format=json&page=1
Parameters:
{ 'filters': 'areaType=overviewareaName=England',
  'format': 'json',
  'page': 1,
  'structure': '{"males":"maleCases","females":"femaleCases"}'}


**(C) 2020 Fabrizio Smeraldi** ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)), all rights reserved.